In [7]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 1. 설정
n_samples = 20000
np.random.seed(42)

# 시간 생성 (초 단위 데이터 가정)
dates = pd.date_range(start='2024-01-01', periods=n_samples, freq='S')

# 2. Random Walk로 기준 가격(Mid Price) 생성
returns = np.random.normal(0, 0.0001, n_samples)
price_path = 100 * np.exp(np.cumsum(returns)) # 100불에서 시작

# 3. 호가창(Order Book) 데이터 생성
# Spread: 매도호가 - 매수호가
spread = np.random.gamma(1, 0.05, n_samples) 
mid_price = price_path

bid_price = mid_price - (spread / 2)
ask_price = mid_price + (spread / 2)

# 주문량 (Volume) 생성 - 핵심 Feature
# 가격이 오를 땐 매수 잔량(Bid Size)이 많아지도록 패턴 주입 (Strong Signal)
signal = np.roll(returns, -10) # 10초 뒤의 수익률을 미리 좀 반영 (Label Leakage 아님, 인과관계 모사)
bid_size = np.random.randint(1, 100, n_samples) + (signal > 0) * 50 
ask_size = np.random.randint(1, 100, n_samples) + (signal < 0) * 50

# 쓰레기 변수 (Feature Selection 연습용)
noise_col_1 = np.random.rand(n_samples)
noise_col_2 = np.random.normal(0, 1, n_samples)

# DataFrame 생성
df = pd.DataFrame({
    'timestamp': dates,
    'bid_price': bid_price,
    'ask_price': ask_price,
    'bid_size': bid_size,
    'ask_size': ask_size,
    'noise_1': noise_col_1,
    'noise_2': noise_col_2
})

# Target 생성: 60초(1분) 뒤의 Mid Price 수익률 예측 (단위: basis point)
# Future Return = (Mid_Price_t+60 / Mid_Price_t) - 1
df['mid_price'] = (df['bid_price'] + df['ask_price']) / 2
df['target_return'] = df['mid_price'].shift(-60) / df['mid_price'] - 1
df['target_return'] = df['target_return'] * 10000 # bps 단위로 변환

# NaN 제거 (마지막 60개 데이터는 미래가 없으므로 삭제)
df = df.dropna()

# 파일 저장 시뮬레이션
df.to_csv('crypto_lob_data.csv', index=False)
print("호가창 데이터 생성 완료: crypto_lob_data.csv")

호가창 데이터 생성 완료: crypto_lob_data.csv


/var/folders/qz/5dkq_4wj0yq8v0bd99hhn7g00000gn/T/ipykernel_50782/1170620587.py:10: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  dates = pd.date_range(start='2024-01-01', periods=n_samples, freq='S')


In [3]:
raw_df = pd.read_csv('crypto_lob_data.csv')

In [25]:
raw_df['timestamp'] > '2024-01-05'

0        False
1        False
2        False
3        False
4        False
         ...  
19935    False
19936    False
19937    False
19938    False
19939    False
Name: timestamp, Length: 19940, dtype: bool

In [53]:
import datetime
raw_df['timestamp'] = pd.to_datetime(raw_df['timestamp'])
raw_df = raw_df.sort_values(by='timestamp')

In [63]:
feature_list = list(raw_df.columns)
feature_list.remove('target_return')

In [73]:
train_end = int(len(raw_df) * 0.8)
vali_end = int(len(raw_df)*0.9)
X_train, Y_train = raw_df[feature_list][:train_end], raw_df['target_return'][:train_end]
X_vali, Y_vali = raw_df[feature_list][train_end:vali_end], raw_df['target_return'][train_end:vali_end]
X_test, Y_test = raw_df[feature_list][vali_end:], raw_df['target_return'][vali_end:]
